In [188]:
import requests
import chess
import chess.pgn
import json
from chess.engine import Cp

In [189]:
url = "https://explorer.lichess.ovh/lichess"

params = {
    "variant": "standard",
    "speeds": "blitz,rapid,classical",
    "ratings": "2200,2500",
    "fen": None
}

In [190]:
game = chess.pgn.Game()

game.headers["Event"] = "Example"

In [191]:
node = game.add_variation(chess.Move.from_uci("g2g4"))

In [192]:
params["fen"] = node.board().fen()

In [193]:
response = requests.get(url, params=params)

In [194]:
tree = json.loads(response.content.decode())

In [195]:
tree['moves'][0]['uci']

'd7d5'

In [196]:
node.ply()

1

In [197]:
def get_cloud_eval(node):
    
    params_eval['fen'] = node.board().fen()
    response = requests.get(url_eval, params=params_eval)
    tree = json.loads(response.content.decode())
        
    return tree['pvs'][0]['cp']

In [198]:
def compute_evaluations(node, tree_move, total_games):
        
    w = tree_move['white']
    b = tree_move['black']
    d = tree_move['draws']

    tot = w+b+d

    tree_move['white'] = (int)(w/tot * 100)
    tree_move['black'] = (int)(b/tot * 100)
    tree_move['draws'] = (int)(d/tot * 100)
        
    tree_move['perc'] = (int)((float)(tot)/total_games * 100.)
    
    n = node.add_variation(chess.Move.from_uci(tree_move['uci']))
    tree_move['eval'] = get_cloud_eval(n)/100.
        

In [209]:
def filter_moves(bIsPlayerTurn, move_list, perc_cumulative):
    """filtra le mosse per trovare o le candidate per il giocatore oppure quelle da considerare dagli avversari

    Args:
        bIsPlayerTurn (bool): si ricerca la mossa del giocatore ?
        move_list (list): tree['moves']
        perc_cumulative (int): nel caso in cui si debbano considerare le mosse avversarie si vuole comprire una % totale non inferiore a questo numero

    Returns:
        list: lista delle mosse candidate e loro relativi commenti
    """
    
    ret_moves = []
    
    
    
    return ret_moves

In [206]:
total_games = tree['white'] + tree['draws'] + tree['black']

for move in tree['moves']:
    compute_evaluations(node, move, total_games)
    
    
eval_sorted_moves = sorted(tree['moves'], key=lambda x: x["eval"])

for i, item in enumerate(tree['moves']):
    item["freq_pos"] = i
    item["eval_pos"] = eval_sorted_moves.index(item)
    item["relative_choid_diff"] = item["eval_pos"] - item["freq_pos"]

In [207]:
for move in tree['moves']:
    print(move['san'], "\t", move['freq_pos'], "\t", move['eval_pos'], "\t", move["relative_choid_diff"])

d5 	 0 	 0 	 0
e5 	 1 	 1 	 0
d6 	 2 	 5 	 3
c5 	 3 	 2 	 -1
h5 	 4 	 7 	 3
g6 	 5 	 8 	 3
c6 	 6 	 6 	 0
e6 	 7 	 4 	 -3
Nf6 	 8 	 10 	 2
Nc6 	 9 	 3 	 -6
b6 	 10 	 9 	 -1
g5 	 11 	 11 	 0


In [204]:
print('\U0001F534', '\U0001F7E0', '\U0001F7E1', '\U0001F7E2')

🔴 🟠 🟡 🟢
